<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/01oai_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [1]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def use_responses(temp):
  response = client.responses.create(
      model="gpt-4.1-mini",
      temperature=temp,
      input="Write a one sentence about Artificial Intelligence."
  )

  return response

In [ ]:
response = use_responses(2)
print(response)
print(type(response))

In [ ]:
print(response.output_text)

`output_text` is a convenience property and is a core concept not only for OpenAI APIs, but for modern SDK design in general. It automatically Traverses the output structure, Extracts all output_text, Concatenates them, A convenience property is a shortcut provided by a library to make your life easier. So, instead of digging through nested objects, writing loops, checking types, the SDK does that work and gives you the most commonly needed value directly.

## Text Generation

Prompting involves crafting effective instructions to guide models toward consistent, desired outputs. Since model responses are non-deterministic, it combines art and science, using proven techniques and best practices.
The instructions parameter is used to give the model high-level behavioral guidance—such as tone, goals, and response style—that has higher priority than prompts in the input. It works alongside message roles to control how the model responds.

[Instructions Details](https://model-spec.openai.com/2025-02-12.html#chain_of_command)

In [ ]:
def use_instructions(inst):
  response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    instructions=inst,
    input="Explain about Generative AI?",
    max_output_tokens=500
  )
  return response.output_text

In [ ]:
print(use_instructions("Talk like an Novice."))

In [ ]:
print(use_instructions("Talk like an Novice but the answer should be very concise."))

In [ ]:
print(use_instructions("Talk like an Novice but the answer should be very concise within 40 words."))

OpenAI’s Responses API uses a clear role hierarchy to control behavior and safety. The system role is managed by OpenAI to enforce policies and alignment. The instruction role defines the assistant’s identity and overall behavior. The developer role sets application-specific rules and constraints. The user role provides end-user input or data to be processed. The assistant role is the model’s generated response. This separation ensures secure, predictable, and well-controlled AI applications.

In [ ]:
instructions = """
You are an academic evaluator.
- Be concise
- Use bullet points
- Give marks out of 10
"""

input = [
    {"role": "user", "content": "Explain Newton’s First Law."}
]

response = client.responses.create(
    model="gpt-4.1-mini",
    temperature=1.6,
    instructions=instructions,
    input=input
)


In [ ]:
print(response.output_text)

In [ ]:
instructions = """
You are a physics examiner.

Evaluation rules:
- Score the answer out of 5 marks
- Identify correctness and missing points
- Give brief feedback
- Do NOT re-explain the topic
- Output format:
  Score: X/5
  Feedback: <one paragraph>
"""


In [ ]:
student_answer = """
Ohm’s Law states that voltage is directly proportional to current.
"""

In [ ]:
student_answer = """
Ohm's law relates voltage and current.
"""

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    instructions=instructions,
    input=[
        {
            "role": "developer",
            "content": "Question: Explain Ohm’s Law."
        },
        {
            "role": "user",
            "content": f"Student answer: {student_answer}"
        }
    ]
)
print(response.output_text)

Score: 1/5
Feedback: The answer correctly identifies that Ohm’s Law relates voltage and current, but it is very incomplete and lacks the essential detail that voltage is proportional to current with resistance as the constant of proportionality (V = IR). More explanation and the correct formula are needed to fully answer the question.


In [ ]:
instructions = """
You are a ROS2 teacher.
"""

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    instructions=instructions,
    input=[
        {
            "role": "developer",
            "content": "Explain concepts using real-world control system examples. Keep it under 150 words"
        },
        {
            "role": "user",
            "content": "Explain PID control simply"
        }
    ]
)

print(response.output_text)

PID control is like adjusting the temperature of your home’s heater.  
- **P (Proportional):** If the room is colder than your target, the heater works harder—the bigger the difference, the stronger the heat.  
- **I (Integral):** If the room has been too cold for a while, the heater adds extra warmth to fix that lingering chill.  
- **D (Derivative):** If the temperature is rising fast, the heater slows down to avoid overheating.  

In ROS2 robots, PID helps control motors smoothly, like keeping a robot arm steady or maintaining a drone’s altitude, by constantly adjusting commands based on the error, its history, and how quickly it’s changing.


In [10]:
response = client.responses.create(
    model="gpt-4.1-mini",
    instructions="You ROS2 teacher. You do not know about any other topic",
    input=[
        {
            "role": "system",
            "content": "You are a helpful control systems teacher."
        },
        {
            "role": "developer",
            "content": "Explain concepts simply, using real-world examples."
        },
        {
            "role": "user",
            "content": "Explain about chemical reaction."
        }
    ]
)

print(response.output_text)

I’m here to help with ROS2 (Robot Operating System 2) and control systems concepts! Chemical reactions are more related to chemistry, but I can give you a simple explanation related to control systems if you'd like.

In control systems, think of a **chemical reaction** like a process where inputs change the system’s state. For example, in a reactor vessel, chemicals mix and react, and the concentration changes over time – this can be modeled and controlled with sensors and actuators.

**Real-world analogy:**

Imagine a coffee machine (control system). You put in water (input), it heats up, and coffee comes out (output). Inside, the heating process is like a reaction—it changes water temperature over time, and a thermostat controls the heat to get the right temperature.

If you'd like, I can explain how to model and control such processes using ROS2 and control theory. Would you like me to?
